In [3]:
# | hide
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [1]:
import polars as pl
from discontinuitypy.datasets import IDsDataset
from discontinuitypy.utils.basic import resample
from beforerr.polars import pl_norm

from sunpy.time import TimeRange
from datetime import timedelta
from space_analysis.utils.speasy import Variables

08-Feb-24 11:23:53: UserWarning: Traceback (most recent call last):
  File "/Users/zijin/micromamba/envs/psp_conjunction/lib/python3.11/site-packages/pdpipe/__init__.py", line 85, in <module>
    from . import skintegrate
  File "/Users/zijin/micromamba/envs/psp_conjunction/lib/python3.11/site-packages/pdpipe/skintegrate.py", line 20, in <module>
    from sklearn.base import BaseEstimator
ModuleNotFoundError: No module named 'sklearn'


08-Feb-24 11:23:53: UserWarning: pdpipe: Scikit-learn or skutil import failed. Scikit-learn-dependent pipeline stages will not be loaded.

08-Feb-24 11:23:53: UserWarning: Traceback (most recent call last):
  File "/Users/zijin/micromamba/envs/psp_conjunction/lib/python3.11/site-packages/pdpipe/__init__.py", line 105, in <module>
    from . import nltk_stages
  File "/Users/zijin/micromamba/envs/psp_conjunction/lib/python3.11/site-packages/pdpipe/nltk_stages.py", line 19, in <module>
    import nltk
ModuleNotFoundError: No module named 'nltk'


08-Feb-2

In [2]:
start = "2019-04-06T12:00"
start = "2019-04-07T01:00"
end = "2019-04-07T12:00"

earth_start = "2019-04-09"
earth_end = "2019-04-12"

tau = timedelta(seconds=10)
ts = timedelta(seconds=1 / 180)

In [3]:
psp_timerange = TimeRange(start, end)
timerange_earth = TimeRange(earth_start, earth_end)

In [4]:
def validate(timerange):
    if isinstance(timerange, TimeRange):
        return [timerange.start.to_string(), timerange.end.to_string()]

In [5]:
psp_mag_vars = Variables(
    dataset="PSP_FLD_L2_MAG_RTN",
    parameters=["psp_fld_l2_mag_RTN"],
    timerange=validate(psp_timerange),
)

psp_mag_data = psp_mag_vars.to_polars()

PSP plasma data


In [6]:
dataset = "PSP_SWP_SPI_SF00_L3_MOM"
parameters = ["DENS", "VEL_RTN_SUN", "TEMP", "SUN_DIST"]
vec_cols = ["Vx RTN", "Vy RTN", "Vz RTN"]

psp_plasma_var = Variables(
    dataset=dataset, parameters=parameters, timerange=validate(psp_timerange)
)

In [7]:
psp_plasma_data = (
    psp_plasma_var.to_polars()
    .with_columns(plasma_speed=pl_norm(vec_cols))
    .rename(
        {
            "Density": "plasma_density",
            "Temperature": "plasma_temperature",
        }
    )
)

# psp_plasma_tau = psp_plasma_data.pipe(resample, every=tau)

In [8]:
psp_ids_dataset = (
    IDsDataset(
        mag_data=psp_mag_data,
        plasma_data=psp_plasma_data,
        tau=tau,
        ts=ts,
        bcols = ["B_R",	"B_T",	"B_N"],
        density_col="plasma_density",
        vec_cols=vec_cols,
        temperature_col="plasma_temperature",
    )
    .find_events()
    .update_candidates_with_plasma_data()
    .export("../data/psp_ids_dataset.arrow")
)

08-Feb-24 11:23:58: UserWarning: Ray execution environment not yet initialized. Initializing...
To remove this warning, run the following python code before doing dataframe operations:

    import ray
    ray.init()


2024-02-08 11:24:00,841	INFO worker.py:1724 -- Started a local Ray instance.
08-Feb-24 11:24:02: UserWarning: Distributing <class 'pandas.core.frame.DataFrame'> object. This may take some time.



Distributing Dataframe:   0%           Elapsed time: 00:00, estimated remaining time: ?

(_deploy_ray_func pid=7929) UserWarning: Traceback (most recent call last):
(_deploy_ray_func pid=7929)   File "/Users/zijin/micromamba/envs/psp_conjunction/lib/python3.11/site-packages/pdpipe/__init__.py", line 85, in <module>
(_deploy_ray_func pid=7929)     from . import skintegrate
(_deploy_ray_func pid=7929)   File "/Users/zijin/micromamba/envs/psp_conjunction/lib/python3.11/site-packages/pdpipe/skintegrate.py", line 20, in <module>
(_deploy_ray_func pid=7929)     from sklearn.base import BaseEstimator
(_deploy_ray_func pid=7929) ModuleNotFoundError: No module named 'sklearn'
(_deploy_ray_func pid=7929) 
(_deploy_ray_func pid=7929) UserWarning: pdpipe: Scikit-learn or skutil import failed. Scikit-learn-dependent pipeline stages will not be loaded.
(_deploy_ray_func pid=7929) UserWarning: Traceback (most recent call last):
(_deploy_ray_func pid=7929)   File "/Users/zijin/micromamba/envs/psp_conjunction/lib/python3.11/site-packages/pdpipe/__init__.py", line 105, in <module>
(_deploy_

Estimated completion of line 12:   0%           Elapsed time: 00:00, estimated remaining time: ?

#### Check

In [9]:
psp_ids_dataset

IDsDataset(name=None, events=shape: (172, 85)
┌────────────┬───────┬───────────┬───────────┬───┬────────────┬────────────┬───────────┬───────────┐
│ time       ┆ count ┆ B_std     ┆ B_mean    ┆ … ┆ v.ion.chan ┆ T.change   ┆ B.change  ┆ v.Alfven. │
│ ---        ┆ ---   ┆ ---       ┆ ---       ┆   ┆ ge         ┆ ---        ┆ ---       ┆ change    │
│ datetime[n ┆ u32   ┆ f64       ┆ f64       ┆   ┆ ---        ┆ f64        ┆ f64       ┆ ---       │
│ s]         ┆       ┆           ┆           ┆   ┆ f64        ┆            ┆           ┆ f64       │
╞════════════╪═══════╪═══════════╪═══════════╪═══╪════════════╪════════════╪═══════════╪═══════════╡
│ 2019-04-07 ┆ 1465  ┆ 19.343886 ┆ 87.705151 ┆ … ┆ -28.694931 ┆ -9.347061  ┆ -0.581797 ┆ 35.386228 │
│ 01:01:00   ┆       ┆           ┆           ┆   ┆            ┆            ┆           ┆           │
│ 2019-04-07 ┆ 1465  ┆ 23.678484 ┆ 86.906228 ┆ … ┆ -34.678102 ┆ -2.815025  ┆ -1.656227 ┆ 21.984223 │
│ 01:01:05   ┆       ┆           ┆           

In [32]:
psp_ids_dataset.plot()

2024-02-08 11:25:44.035 | INFO     | discontinuitypy.datasets:overview_plot:124 - Overview plot: 2019-04-07 03:36:25 - 2019-04-07 03:36:35
2024-02-08 11:25:44.035 | DEBUG    | discontinuitypy.datasets:overview_plot:125 - n.change: -29.025169372558594, $\Delta v_{ion}$: -17.236806561626793, T.change: 2.6445999145507812, v.Alfven.change: 23.738670036940334


:Layout
   .NdOverlay.I  :NdOverlay   [B comp]
      :Curve   [time]   (B)
   .Curve.I      :Curve   [time]   (plasma_density)
   .NdOverlay.II :NdOverlay   [veloity comp]
      :Curve   [time]   (v)
   .Curve.II     :Curve   [time]   (plasma_temperature)

In [11]:
# Resampled dataset

ts = timedelta(seconds=1 / 11)

psp_mag_data_r = psp_mag_data.pipe(resample, every=ts)

In [12]:
psp_mag_data_r.columns

['time', 'B_R', 'B_T', 'B_N']

In [13]:
psp_ids_dataset_r = (
    IDsDataset(
        mag_data=psp_mag_data_r,
        plasma_data=psp_plasma_data,
        tau=tau,
        ts=ts,
        vec_cols=vec_cols,
    )
    .find_events()
    .update_candidates_with_plasma_data()
    .export("../data/psp_ids_dataset_r.arrow")
)

08-Feb-24 11:24:06: UserWarning: Distributing <class 'pandas.core.frame.DataFrame'> object. This may take some time.



Distributing Dataframe:   0%           Elapsed time: 00:00, estimated remaining time: ?

(_deploy_ray_func pid=7931) UserWarning: Distributing <class 'dict'> object. This may take some time.
Distributing Dataframe: 100%██████████ Elapsed time: 00:00, estimated remaining time: 00:00
(_deploy_ray_func pid=7931) UserWarning: Distributing <class 'dict'> object. This may take some time.
Distributing Dataframe: 100%██████████ Elapsed time: 00:00, estimated remaining time: 00:00
(_deploy_ray_func pid=7931) UserWarning: Distributing <class 'dict'> object. This may take some time.
Distributing Dataframe: 100%██████████ Elapsed time: 00:00, estimated remaining time: 00:00
(_deploy_ray_func pid=7931) UserWarning: Distributing <class 'dict'> object. This may take some time.
Distributing Dataframe: 100%██████████ Elapsed time: 00:00, estimated remaining time: 00:00
(_deploy_ray_func pid=7931) UserWarning: Distributing <class 'dict'> object. This may take some time.
Distributing Dataframe: 100%██████████ Elapsed time: 00:00, estimated remaining time: 00:00
(_deploy_ray_func pid=7931) Us

Estimated completion of line 9:   0%           Elapsed time: 00:00, estimated remaining time: ?

## Wind dataset


In [14]:
import astropy.units as u
from astropy.constants import k_B, m_p

In [15]:
def thermal_spd2temp(speed, speed_unit=u.km / u.s):
    return (m_p * (speed * speed_unit) ** 2 / 2).to("eV").value


def df_thermal_spd2temp(df: pl.LazyFrame, speed_col, speed_unit=u.km / u.s):
    df = df.collect()
    return df.with_columns(
        plasma_temperature=thermal_spd2temp(df[speed_col].to_numpy(), speed_unit)
    ).lazy()


thermal_spd2temp(1)

0.005219842457426576

In [16]:
Variables(
    dataset="WI_K0_SWE",
    parameters=["V_GSM", "Np", "THERMAL_SPD"],
    timerange=validate(timerange_earth),
).preview()

VX (GSM),VY (GSM),VZ (GSM),time,Ion Np,SW Vth
f64,f64,f64,datetime[ns],f64,f64
-413.968323,-15.499876,-16.377909,2019-04-09 00:03:02.175,3.848233,26.357391
-408.129852,-21.568832,3.775948,2019-04-09 00:04:39.701,4.287208,29.379347
-406.557678,0.265325,-19.682274,2019-04-09 00:06:12.652,4.911876,29.542917
-410.871796,-11.287542,-20.01589,2019-04-09 00:07:50.213,4.266828,30.832428
-414.371246,-10.226903,-26.906651,2019-04-09 00:09:36.926,3.35119,24.394451


In [19]:
wind_plasma_vars = Variables(
    dataset="WI_K0_SWE",
    parameters=["V_GSM", "Np", "THERMAL_SPD"],
    timerange=validate(timerange_earth),
)

In [20]:
ts = timedelta(seconds=1 / 11)
tau = timedelta(seconds=30)

wind_vec_cols = ["VX (GSM)", "VY (GSM)", "VZ (GSM)"]


wind_mag_data = Variables(
    dataset="WI_H2_MFI",
    parameters=["BGSM"],
    timerange=validate(timerange_earth),
).to_polars()

wind_plasma_data = (
    wind_plasma_vars.to_polars()
    .with_columns(
        plasma_speed=pl_norm(wind_vec_cols),
    )
    .pipe(df_thermal_spd2temp, "SW Vth")
    .rename(
        {
            "Ion Np": "plasma_density",
        }
    )
)

In [21]:
wind_ids_dataset = (
    IDsDataset(
        mag_data=wind_mag_data,
        plasma_data=wind_plasma_data,
        tau=tau,
        ts=ts,
        vec_cols=wind_vec_cols,
        density_col="plasma_density",
        temperature_col="plasma_temperature",
    )
    .find_events()
    .update_candidates_with_plasma_data()
    .export("../data/wind_ids_dataset_r.arrow")
)

08-Feb-24 11:24:20: UserWarning: Distributing <class 'pandas.core.frame.DataFrame'> object. This may take some time.



Distributing Dataframe:   0%           Elapsed time: 00:00, estimated remaining time: ?

Estimated completion of line 11:   0%           Elapsed time: 00:00, estimated remaining time: ?

In [28]:
wind_ids_dataset.plot(type="overview")

2024-02-08 11:25:22.361 | INFO     | discontinuitypy.datasets:overview_plot:124 - Overview plot: 2019-04-10 22:16:15 - 2019-04-10 22:16:45
2024-02-08 11:25:22.362 | DEBUG    | discontinuitypy.datasets:overview_plot:125 - n.change: -0.06765460968017578, $\Delta v_{ion}$: -17.891144252586855, T.change: 0.8499286297780282, v.Alfven.change: -0.24390681645485301


:Layout
   .NdOverlay.I  :NdOverlay   [B comp]
      :Curve   [time]   (B)
   .Curve.I      :Curve   [time]   (plasma_density)
   .NdOverlay.II :NdOverlay   [veloity comp]
      :Curve   [time]   (v)
   .Curve.II     :Curve   [time]   (plasma_temperature)